In [11]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [13]:
%autoreload
from pathlib import Path
import sys
root = Path().resolve().absolute().parent.parent
print(root)
sys.path.append(str(root))

from src.pipeline import Pipeline, BuilingIdsEnum
pipe = Pipeline()



/Users/matsalexander/Desktop/SolarEnergyImpact


In [14]:
building_b = pipe.get_data(BuilingIdsEnum.B)
building_c = pipe.get_data(BuilingIdsEnum.C)
building_a = pipe.get_data(BuilingIdsEnum.A)

# from november 1. to 1. mars only use data from dataset b 
# else use data from dataset c and a

# remove from 2024-01-15 from dataset A
mask_b = (building_b['timestamp'] >= '2024-01-15') & (building_b['timestamp'] <= '2024-01-15')
building_b = building_b[~mask_b]

start_winter = '2023-11-01'
end_winter = '2024-03-01'
mask_b = (building_b['timestamp'] >= start_winter) & (building_b['timestamp'] <= end_winter)
building_b = building_b[mask_b]

mask_c = (building_c['timestamp'] < start_winter) | (building_c['timestamp'] > end_winter)
building_c = building_c[mask_c]

# building a, only before 2023-09-17.
end_dataset_a = '2023-09-17'
mask_a = (building_a['timestamp'] < end_dataset_a)
building_a = building_a[mask_a]



# normalize the value_import by area and call it value for main and building_b
building_b['value'] = building_b['value_import'] / building_b['area']
building_c['value'] = building_c['net_consumption'] / building_c['area']
building_a['value'] = building_a['value_import'] / building_a['area']


combined_df = pd.concat([building_a, building_b, building_c], ignore_index=True)
# reset index
combined_df = combined_df.reset_index(drop=True)
# Perform the train-test split with stratification based on 'building_id'
train_data, test_data = train_test_split(
    combined_df,
    test_size=0.20,
    random_state=42,
    stratify=combined_df['building']
)

# select features
target = "value"
features = ["timestamp", "temperature"] #, "cloud_fraction", "precipitation", "area", "wind_speed"

train_data = train_data[features + [target]]
test_data = test_data[features + [target]]



In [15]:
# Train the model using AutoGluon
predictor = TabularPredictor(label=target, eval_metric='mean_absolute_error').fit(
    train_data, 
    presets='best_quality',
    excluded_model_types=['KNN']
    )

No path specified. Models will be saved in: "AutogluonModels/ag-20241113_193908"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.6.0: Mon Jul 29 21:14:30 PDT 2024; root:xnu-10063.141.2~1/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       13.50 GB / 32.00 GB (42.2%)
Disk Space Avail:   607.65 GB / 926.35 GB (65.6%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal 

[1000]	valid_set's l1: 0.00495001
[2000]	valid_set's l1: 0.00467688
[3000]	valid_set's l1: 0.00451896
[4000]	valid_set's l1: 0.00441288
[5000]	valid_set's l1: 0.00433018
[6000]	valid_set's l1: 0.00426813
[7000]	valid_set's l1: 0.00421587
[8000]	valid_set's l1: 0.00418395
[9000]	valid_set's l1: 0.00415196
[10000]	valid_set's l1: 0.00413354
[1000]	valid_set's l1: 0.00482558
[2000]	valid_set's l1: 0.00456458
[3000]	valid_set's l1: 0.00442977
[4000]	valid_set's l1: 0.00435085
[5000]	valid_set's l1: 0.0043035
[6000]	valid_set's l1: 0.00426043
[7000]	valid_set's l1: 0.00423126
[8000]	valid_set's l1: 0.0042121
[9000]	valid_set's l1: 0.00419536
[10000]	valid_set's l1: 0.00418205
[1000]	valid_set's l1: 0.00495874
[2000]	valid_set's l1: 0.00471132
[3000]	valid_set's l1: 0.00457823
[4000]	valid_set's l1: 0.00448806


	Ran out of time, early stopping on iteration 4903. Best iteration is:
	[4903]	valid_set's l1: 0.0044302


[1000]	valid_set's l1: 0.00480689
[2000]	valid_set's l1: 0.00456495
[3000]	valid_set's l1: 0.00442552
[4000]	valid_set's l1: 0.00433315


	Ran out of time, early stopping on iteration 4091. Best iteration is:
	[4073]	valid_set's l1: 0.00432586


...

	Ran out of time, early stopping on iteration 5645. Best iteration is:
	[5631]	valid_set's l1: 0.00441392
	-0.0044	 = Validation score   (-mean_absolute_error)
	573.17s	 = Training   runtime
	1.23s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 24.08s of the 324.19s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 141. Best iteration is:
	[141]	valid_set's l1: 0.00463086
	Ran out of time, early stopping on iteration 145. Best iteration is:
	[145]	valid_set's l1: 0.00444555
	Ran out of time, early stopping on iteration 151. Best iteration is:
	[151]	valid_set's l1: 0.0045299
	Ran out of time, early stopping on iteration 155. Best iteration is:
	[154]	valid_set's l1: 0.00432945
	Ran out of time, early stopping on iteration 166. Best iteration is:
	[164]	valid_set's l1: 0.00439823
	Ran out of time, early stopping on iteration 157. Best iteration is:
	[

[1000]	valid_set's l1: 0.00484199
[2000]	valid_set's l1: 0.00457062
[3000]	valid_set's l1: 0.00444799
[4000]	valid_set's l1: 0.0043723
[5000]	valid_set's l1: 0.00432884
[6000]	valid_set's l1: 0.00428791
[7000]	valid_set's l1: 0.00426231
[8000]	valid_set's l1: 0.00424727
[9000]	valid_set's l1: 0.00423461
[10000]	valid_set's l1: 0.00421675
[1000]	valid_set's l1: 0.00503942
[2000]	valid_set's l1: 0.00476434
[3000]	valid_set's l1: 0.0046126
[4000]	valid_set's l1: 0.00450822
[5000]	valid_set's l1: 0.00443723
[6000]	valid_set's l1: 0.00439485
[7000]	valid_set's l1: 0.00436178
[8000]	valid_set's l1: 0.0043363
[9000]	valid_set's l1: 0.00431617
[10000]	valid_set's l1: 0.00429268
[1000]	valid_set's l1: 0.00487765
[2000]	valid_set's l1: 0.00460555
[3000]	valid_set's l1: 0.00445573
[4000]	valid_set's l1: 0.00436439
[5000]	valid_set's l1: 0.00430051
[6000]	valid_set's l1: 0.0042445
[7000]	valid_set's l1: 0.00420249
[8000]	valid_set's l1: 0.0041662
[9000]	valid_set's l1: 0.00413638
[10000]	valid_set

	-0.0042	 = Validation score   (-mean_absolute_error)
	1266.33s	 = Training   runtime
	2.33s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 531.7s of the 1433.77s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's l1: 0.00410531
[2000]	valid_set's l1: 0.00407824
[1000]	valid_set's l1: 0.00428664
[2000]	valid_set's l1: 0.00425965
[1000]	valid_set's l1: 0.00406805
[2000]	valid_set's l1: 0.00400745
[1000]	valid_set's l1: 0.00406964
[2000]	valid_set's l1: 0.00406472
[1000]	valid_set's l1: 0.00425099
[2000]	valid_set's l1: 0.00419202
[3000]	valid_set's l1: 0.00418371
[4000]	valid_set's l1: 0.00418067
[1000]	valid_set's l1: 0.00402891
[2000]	valid_set's l1: 0.00401465
[1000]	valid_set's l1: 0.00424272
[2000]	valid_set's l1: 0.00419308
[3000]	valid_set's l1: 0.00418739
[1000]	valid_set's l1: 0.00424935
[2000]	valid_set's l1: 0.00421274


	-0.0041	 = Validation score   (-mean_absolute_error)
	160.69s	 = Training   runtime
	0.4s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 369.9s of the 1271.97s of remaining time.
	-0.0033	 = Validation score   (-mean_absolute_error)
	1.61s	 = Training   runtime
	0.33s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 367.68s of the 1269.76s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 6225.
	Ran out of time, early stopping on iteration 6315.
	Ran out of time, early stopping on iteration 6300.
	Ran out of time, early stopping on iteration 6481.
	Ran out of time, early stopping on iteration 6813.
	Ran out of time, early stopping on iteration 6920.
	Ran out of time, early stopping on iteration 7302.
	Ran out of time, early stopping on iteration 8418.
	-0.0041	 = Validation score   (-mean_absolute_error)
	352.

In [16]:
# AutogluonModels/ag-
# Evaluate on test data
performance = predictor.evaluate(test_data)
# best model: ag-20241022_161331

print("Evaluation Performance:")
performance
# reset index 
test_data = test_data.reset_index(drop=True)
# To see feature importance
global_importance = predictor.feature_importance(test_data)
print("\nFeature Importance:")
print(global_importance)  # Shows which features had the most impact on model predictions


Computing feature importance via permutation shuffling for 2 features using 2448 rows with 5 shuffle sets...


Evaluation Performance:


	205.8s	= Expected runtime (41.16s per shuffle set)
	90.68s	= Actual runtime (Completed 5 of 5 shuffle sets)



Feature Importance:
             importance    stddev       p_value  n  p99_high   p99_low
timestamp      0.005104  0.000103  2.021936e-08  5  0.005317  0.004891
temperature    0.005102  0.000160  1.170895e-07  5  0.005433  0.004772


In [17]:
performance

{'mean_absolute_error': -0.003147551549112532,
 'root_mean_squared_error': -0.004858080780383512,
 'mean_squared_error': -2.3600948868731676e-05,
 'r2': 0.7072616613183256,
 'pearsonr': 0.8416842568799808,
 'median_absolute_error': -0.0016578313675645294}

In [18]:
# model location => AutogluonModels/ag-20241016_095906
main_building = pipe.get_data(BuilingIdsEnum.MAIN)

data_predict = main_building[features]
data_predict


,timestamp,temperature
0,2023-07-01 00:00:00,13.6
1,2023-07-01 01:00:00,13.2
2,2023-07-01 02:00:00,12.3
3,2023-07-01 03:00:00,11.9
4,2023-07-01 04:00:00,11.9
...,...,...
10338,2024-09-03 18:00:00,17.8
10339,2024-09-03 19:00:00,17.8
10340,2024-09-03 20:00:00,17.7
10341,2024-09-03 21:00:00,18.0


In [19]:
prediciton1 = predictor.predict(data_predict)

In [21]:
# save predicitons as a csv in data folder from root.
import datetime
from pathlib import Path


prediciton1_df = pd.DataFrame(prediciton1)
date_time = datetime.datetime.now().strftime("%Y%m%d_%H")
my_path = Path().resolve().parent.parent / 'data'/ "pred" / 'prediction_winter.csv'
if my_path.exists():
    my_path = Path().resolve().parent.parent / 'data'/ "pred" / f'prediction_winter2_{date_time}.csv'
# create folder
my_path.parent.mkdir(parents=True, exist_ok=True)
prediciton1_df.to_csv(my_path, index=False)